# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Air Quality** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book
HopsworksSettings initialized!


In [2]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs.airquality import util
import os

In [3]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

datetime.datetime(2025, 11, 17, 17, 51, 49, 437772)

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [4]:
project = hopsworks.login(engine="python", project="air_quality_prediction")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
# location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
# location = json.loads(location_str)
# country=location['country']
# city=location['city']
# street=location['street']

sensor_secret_names = [
    "SENSOR_LOCATION_bankgatan_JSON",
    "SENSOR_LOCATION_linakersvagen_JSON",
    "SENSOR_LOCATION_trollebergsvagen_JSON",
]

sensors = {}
for name in sensor_secret_names:
    data = json.loads(secrets.get_secret(name).value)
    sensors[data["street"]] = {
        "country":   data["country"],
        "city":      data["city"],
        "street":    data["street"],
        "latitude":  data["latitude"],
        "longitude": data["longitude"],
        "aqicn_url": data["aqicn_url"],
    }

2025-11-17 17:51:49,586 INFO: Initializing external client
2025-11-17 17:51:49,587 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-17 17:51:50,314 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-11-17 17:51:51,397 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271977


In [5]:
from sklearn.preprocessing import LabelEncoder


air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1
)

air_quality_df_full = air_quality_fg.read()
streets_sorted = sorted(air_quality_df_full['street'].dropna().unique().tolist())
street_encoder = LabelEncoder().fit(streets_sorted)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.97s) 


## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [6]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=2,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2025-11-17 17:52:00,373 INFO: Initializing for batch retrieval of feature vectors


Downloading: 0.000%|          | 0/171014 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/111397 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/77018 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/108093 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/19961 elapsed<00:00 remaining<?

In [7]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='8.042732E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'int'],
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style="color:#ff5f27;">✨ Get Weather Forecast Features with Feature View   </span>



In [ ]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)




### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [9]:
import datetime
today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions',
    description='Air Quality prediction monitoring',
    version=1,
    primary_key=['city','street','date','days_before_forecast_day'],
    event_time="date",
)

monitor_fg

In [ ]:
all_preds_for_city = []  

for street, meta in sensors.items():

    batch_data = weather_fg.filter(weather_fg.date >= today).read().copy()
    if batch_data.empty:
        print(f"[WARN] No weather rows for {street}; skipping.")
        continue

    batch_data['country'] = meta['country']
    batch_data['city']    = meta['city']
    batch_data['street']  = meta['street']

    batch_data['date'] = pd.to_datetime(batch_data['date']).dt.tz_localize(None)
    batch_data = batch_data.sort_values('date').reset_index(drop=True)
    batch_data['days_before_forecast_day'] = ((batch_data['date'] - today).dt.days + 1)
    batch_data = batch_data.query('days_before_forecast_day >= 1')

    batch_data['street_encode'] = street_encoder.transform(batch_data['street']).astype('float32')

    hist_aq = air_quality_fg.filter(
        (air_quality_fg.street == street) & 
        (air_quality_fg.date < today)
    ).read()
    
    if hist_aq.empty:
        print(f"[WARN] No historical air quality for {street}; skipping.")
        continue
    
    hist_aq['date'] = pd.to_datetime(hist_aq['date']).dt.tz_localize(None)
    hist_aq = hist_aq.sort_values('date').tail(3).reset_index(drop=True)
    
    if len(hist_aq) < 3:
        print(f"[WARN] Less than 3 days of history for {street}; skipping.")
        continue

    predictions = []
    
    for idx, row in batch_data.iterrows():
        day_num = row['days_before_forecast_day']
        
        if day_num == 1:
            lag_1 = hist_aq.iloc[-1]['pm25']
            lag_2 = hist_aq.iloc[-2]['pm25']
            lag_3 = hist_aq.iloc[-3]['pm25']
        elif day_num == 2:
            lag_1 = predictions[0]
            lag_2 = hist_aq.iloc[-1]['pm25']
            lag_3 = hist_aq.iloc[-2]['pm25']
        elif day_num == 3:
            lag_1 = predictions[1]
            lag_2 = predictions[0]
            lag_3 = hist_aq.iloc[-1]['pm25']
        else:
            lag_1 = predictions[day_num - 2]
            lag_2 = predictions[day_num - 3]
            lag_3 = predictions[day_num - 4]
        
        features = [
            row['temperature_2m_mean'],
            row['precipitation_sum'],
            row['wind_speed_10m_max'],
            row['wind_direction_10m_dominant'],
            row['street_encode'],
            lag_1,
            lag_2,
            lag_3
        ]
        
        pred = retrieved_xgboost_model.predict([features])[0]
        predictions.append(pred)

    batch_data['predicted_pm25'] = predictions
    batch_data['pm25_lag_1day'] = [predictions[i-1] if i > 0 else hist_aq.iloc[-1]['pm25'] for i in range(len(predictions))]
    batch_data['pm25_lag_2day'] = [predictions[i-2] if i > 1 else (predictions[0] if i == 1 else hist_aq.iloc[-2]['pm25']) for i in range(len(predictions))]
    batch_data['pm25_lag_3day'] = [predictions[i-3] if i > 2 else (predictions[i-1] if i == 2 else (predictions[0] if i == 1 else hist_aq.iloc[-3]['pm25'])) for i in range(len(predictions))]

    monitor_fg.insert(batch_data.drop(columns=['street_encode']), wait=True)

    all_preds_for_city.append(batch_data.drop(columns=['street_encode']))

In [11]:
print("Timedelta:")
print((batch_data['date'] - today).dt.days + 1)

Timedelta:
0    2
1    3
2    4
3    5
4    6
5    7
6    8
Name: date, dtype: int64


### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

### Create Forecast Graph
Draw a graph of the predictions with dates as a PNG and save it to the github repo
Show it on github pages

In [12]:
from pathlib import Path
import matplotlib.pyplot as plt
root_dir = str(Path().absolute())

from pathlib import Path

# this part is specific to me due to directory naming issues. 
def find_repo_root():
    here = Path.cwd()
    for p in [here, *here.parents]:
        if (p / "docs").exists():
            return p
    return here  

project_root = find_repo_root()

img_dir = project_root / "docs" / "air-quality" / "assets" / "img"
img_dir.mkdir(parents=True, exist_ok=True)

today_str = pd.Timestamp.today().strftime("%Y-%m-%d")
today_str = today.strftime("%Y-%m-%d")

dataset_api = project.get_dataset_api()
if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")



In [13]:
city = next(iter(sensors.values()))['city']  # since all sensors are in the same city
preds_1day_city = monitor_fg.filter(
    (monitor_fg.city == city) &
    (monitor_fg.days_before_forecast_day == 1)
).read().copy()

def _norm_dates(series):
    s = pd.to_datetime(series, errors="coerce")
    try:
        s = s.dt.tz_convert(None)   
    except Exception:
        pass
    return s.dt.normalize()

for street, meta in sensors.items():

    preds_1day = preds_1day_city[preds_1day_city['street'] == street].copy()

    outcomes = air_quality_df_full[
        (air_quality_df_full['city'] == meta['city']) &
        (air_quality_df_full['street'] == street)
    ][['date', 'pm25']].copy()


    try:
        nday_df = next(df for df in all_preds_for_city if df['street'].iloc[0] == street)
    except StopIteration:
        print(f"[WARN] No in-memory forecast DF found for street '{street}'. Skipping plots.")
        continue


    safe_city = str(meta['city']).replace(' ', '_').lower()
    safe_street = str(street).replace(' ', '_').lower()

    pred_file_path = str(img_dir / f"{safe_city}_{safe_street}_pm25_forecast.png")
    hindcast_file_path = str(img_dir / f"{safe_city}_{safe_street}_pm25_hindcast_1day.png")


    print(
        street,
        "rows:", len(nday_df),
        "nans:", nday_df['predicted_pm25'].isna().sum() if 'predicted_pm25' in nday_df.columns else 'col-missing',
        "dates:", nday_df['date'].min(), "→", nday_df['date'].max()
    )
    plt = util.plot_air_quality_forecast(meta['city'], street, nday_df, pred_file_path)
    plt.close()


    if preds_1day.empty or outcomes.empty:
        print(f"[WARN] Empty preds_1day or outcomes for '{street}'. Skipping hindcast plot.")
    else:
  
        preds_1day['date'] = _norm_dates(preds_1day['date'])
        outcomes['date']   = _norm_dates(outcomes['date'])

        min_d, max_d = outcomes['date'].min(), outcomes['date'].max()
        preds_1day = preds_1day[(preds_1day['date'] >= min_d) & (preds_1day['date'] <= max_d)].copy()

        hindcast_df = (
            preds_1day[['date', 'predicted_pm25']]
            .merge(outcomes, on='date', how='inner')
            .sort_values('date')
        )

        if hindcast_df.empty:
            print(f"[WARN] No overlapping dates for hindcast at '{street}'. Skipping hindcast plot.")
        else:
            plt = util.plot_air_quality_forecast(
                meta['city'], street, hindcast_df, hindcast=True, file_path=hindcast_file_path
            )
            plt.close()


    hops_dir = f"Resources/airquality/{safe_city}_{safe_street}_{today_str}"
    dataset_api.upload(pred_file_path, hops_dir, overwrite=True)
    if Path(hindcast_file_path).exists():
        dataset_api.upload(hindcast_file_path, hops_dir, overwrite=True)

print(f"The images are saved here: {project.get_url()}/settings/fb/path/Resources/airquality")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.00s) 
bankgatan rows: 7 nans: 0 dates: 2025-11-18 00:00:00 → 2025-11-24 00:00:00
2025-11-17 18:02:00,124 WARNING: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.



Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_bankgata…

Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_bankgata…

linåkersvägen rows: 7 nans: 0 dates: 2025-11-18 00:00:00 → 2025-11-24 00:00:00


Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_linåkers…

Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_linåkers…

trollebergsvägen rows: 7 nans: 0 dates: 2025-11-18 00:00:00 → 2025-11-24 00:00:00


Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_trollebe…

Uploading /Users/sebastian/Documents/KTH/Year 1/ID2223-SML/mlfs-book/docs/air-quality/assets/img/lund_trollebe…

The images are saved here: https://c.app.hopsworks.ai:443/p/1271977/settings/fb/path/Resources/airquality


### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

__This graph will be empty to begin with - this is normal.__

After a few days of predictions and observations, you will get data points in this graph.

### Upload the prediction and hindcast dashboards (png files) to Hopsworks


---